In [2]:
import math
import inspect
import simpy
# import scipy.stats as scs
from numpy import random
import pandas as pd

In [2]:
def retrieve_name(var):
    callers_local_vars = inspect.currentframe().f_back.f_locals.items()
    return [var_name for var_name, var_val in callers_local_vars if var_val is var][0]

$V = 20 \% 4 + 1 = 1$

Задача 1

В парикмахерской работают **3 мастера**, а в зале ожидания расположены **3 стула**. Поток клиентов имеет интенсивность **12 клиентов в час**. Среднее **время обслуживания составляет 20 мин**. Определить показатели работы СМО.

MMcK Queue - Multiserver, Finite-Capacity System

$\lambda = 12$ - интенсивность потока клиентов

$\mu = 3$ - интенсивность обслуживания

$\rho = \dfrac{\lambda}{\mu} = \dfrac{12}{3} = 4$ -  интенсивность траффика

$ K = 6$ - максимальное кол-во клиентов

$ c = 3$ - кол-во "серверов"

In [246]:
lam = 12
mu = 3
rho = lam / mu

Вероятность попадания в каждое из возможных состояний: 


$$p_0 = \left[ \sum^c_{n=0} \frac{\rho^n}{n!} + \frac{\rho^c}{c!} \sum^{k-c}_{n=1} \left(\frac{\rho}{c}\right)^n  \right]^{-1}$$

<!-- $$p_0 = \left[ \frac{3^0}{0!} + \frac{3^1}{1!} + \frac{3^2}{2!} + \frac{3^2}{2!} \left(\frac{3^1}{2^1}+ \frac{3^2}{2^2}+ \frac{3^3}{2^3}+ \frac{3^4}{2^4} \right) \right]^{-1} = 0,0158 = 0,016$$ -->

$$\begin{equation*}
P_n = 
 \begin{cases}
   \dfrac{\rho^n}{n!}p_0 &\text{,if $n = 1,2,... , c$}\\
   \dfrac{\rho^c}{c!}\left(\dfrac{\rho}{c}\right)^{n-c}p_0 &\text{,if $n = c+1, ..., K$}
 \end{cases}
\end{equation*}$$

In [4]:
def calculate_p0(c, K, rho):
    first_sum = sum([rho**n / math.factorial(n) for n in range(c + 1)])
    second_sum = sum([(rho/c)**n for n in range(1, K - c + 1)])
    return (first_sum + (rho**c / math.factorial(c)) * second_sum)**(-1)

def calculate_Pn(n, rho, c, K, p_0):
    if 1 <= n <= c:
        return (rho**n / math.factorial(n)) * p_0
    elif c + 1 <= n <= K:
        return (rho**c / math.factorial(c)) * (rho/c)**(n - c) * p_0
    else:
        raise ValueError('n has to be an int number from from 1 to K')

In [247]:
c = 3
K = 6
rho = 4
p_0 = calculate_p0(c, K, rho)

# for i in range(1, 7):
#     print(f"p_{i} = calculate_Pn({i}, rho, c, K, p_0)")

p_1 = calculate_Pn(1, rho, c, K, p_0)
p_2 = calculate_Pn(2, rho, c, K, p_0)
p_3 = calculate_Pn(3, rho, c, K, p_0)
p_4 = calculate_Pn(4, rho, c, K, p_0)
p_5 = calculate_Pn(5, rho, c, K, p_0)
p_6 = calculate_Pn(6, rho, c, K, p_0)

prob_list = [p_0, p_1, p_2, p_3, p_4, p_5, p_6]

assert abs(sum(prob_list) - 1) < 10**(-3)

for i, j in enumerate(prob_list):
    print(f"p_{i} = %.4f" % j)

p_0 = 0.0122
p_1 = 0.0487
p_2 = 0.0974
p_3 = 0.1299
p_4 = 0.1732
p_5 = 0.2309
p_6 = 0.3078


# Теор показатели

In [331]:
Notation = {
    "Abs_capacity": {"name": "Абсолютная пропускная способность СМО"},
    "Rel_capacity": {"name": "Относительная пропускная способность СМО"},
    "Mean_busy_cycle": {"name": 'Средняя продолжительность периода занятости СМО'},
    "System_utilization": {"name": "Коэффициент использования СМО"},
    "Q_time_avg": {"name": "Среднее время ожидания заявки в очереди"},
    "Mean_response_time": {"name": "Среднее время пребывания заявки в СМО"},
    "P_reject": {"name": "Вероятность отказа заявке в обслуживании без ожидания"},
    "Q_time_DF": {"name": "Закон распределения времени ожидания заявки в очереди"},
    "P_immediate": {"name": "Вероятность того, что вновь поступившая заявка немедленно будет принята к обcлуживанию"},
    # "": {"name": "Закон распределения времени пребывания заявки в СМО"},
    "Q_num_avg": {'name': 'Среднее число заявок, находящихся в очереди'},
    'N_cust_avg': {'name': 'Среднее число заявок, находящихся в СМО'}
}


1.1. Абсолютная пропускная способность СМО – среднее число заявок

1.2. Относительная пропускная способность СМО 

1.3. Средняя продолжительность периода занятости СМО

1.4. Коэффициент использования СМО – средняя доля времени, в течение которого СМО занята обслуживанием заявок

2.1. Среднее время ожидания заявки в очереди

2.2. Среднее время пребывания заявки в СМО

2.3. Вероятность отказа заявке в обслуживании без ожидания

2.4. Вероятность того, что вновь поступившая заявка немедленно будет принята к обcслуживанию

2.5. Закон распределения времени ожидания заявки в очереди

2.7. Среднее число заявок, находящихся в очереди

$r = \dfrac{\rho}{c}$

Q_num_avg = $\bar Q = \dfrac{\rho^c r p_0}{c!(1-r)^2} \left( 1+(K-c)r^{K-c+1}- (K-c+1)r^{K-c} \right)$

2.8. Среднее число заявок, находящихся в СМО

N_cust_avg = $\bar N = \bar Q + \rho(1 - p_k)$

In [8]:
def calculate_Q_num_avg(rho, c, K, r, p_0):
    Q_num_avg = (
        (rho**c * r * p_0) / (math.factorial(c) * (1 - r)**2)
        * (1 + (K - c)*r**(K - c + 1) - (K - c + 1) * r**(K - c))
    )
    return Q_num_avg

In [334]:
P_reject = p_6

Abs_capacity = lam * (1 - P_reject)

Rel_capacity = (1 - P_reject)

Mean_busy_cycle = (1 - p_0) / (lam * p_0)

System_utilization = 1 - p_0

r = rho / c

Q_num_avg = calculate_Q_num_avg(rho, c, K, r, p_0)

Q_time_avg = Q_num_avg / lam

N_cust_avg = Q_num_avg + rho * (1 - p_6)

Mean_response_time = N_cust_avg / lam

P_immediate = p_0 + p_1 + p_2

In [335]:
for key in Notation.keys():
    if key not in ('Q_time_DF', ):
        try:
            Notation[key]['value'] = globals()[key]
        except KeyError:
            print(key, 'not calculated')

In [336]:
for i, j in Notation.items():
    if j.get('value'):
        print(j['name'], ': ', '%.3f'%j['value'], sep='')

Абсолютная пропускная способность СМО: 8.306
Относительная пропускная способность СМО: 0.692
Средняя продолжительность периода занятости СМО: 6.761
Коэффициент использования СМО: 0.988
Среднее время ожидания заявки в очереди: 0.130
Среднее время пребывания заявки в СМО: 0.361
Вероятность отказа заявке в обслуживании без ожидания: 0.308
Вероятность того, что вновь поступившая заявка немедленно будет принята к обcлуживанию: 0.158
Среднее число заявок, находящихся в очереди: 1.558
Среднее число заявок, находящихся в СМО: 4.327


2.6. Закон распределения времени пребывания заявки в СМО

$F_W(t)=1-\sum\limits_{n=c}^{K-1}\pi_n \sum\limits_{i=0}^{n-c}\dfrac{(c\mu t)^i e^{-c\mu t}}{i!}$

$F_W(0)=p_0+p_1$

In [151]:
# tbd 

# Simulation

In [339]:
from dataclasses import dataclass

@dataclass
class Book:
    title: str
    author: str

In [350]:
env.run()

In [398]:
class MMcK_Queue:
    def __init__(self, env, lambda_, mu, c, K, seed=None):
        self.env = env
        self.lambda_ = lambda_
        self.mu = mu
        # self.K = K 
        self.queue = simpy.Store(env, capacity=K - c)
        self.server = simpy.Resource(self.env, capacity=c)
        self.current_customer = None # create dataclass??
        self.customer_visits_data = []
        # self.metrics = {}

    
    def run_until_time(self, runtime=10):

        self.env.process(self.generate_arrivals_forever())
        self.env.run(until=runtime)
        # return self.results
    
    
    def generate_arrivals_forever(self):
        self.current_customer = id_num = 0
        while True:
            # wait for the next customer
            yield self.env.timeout(random.exponential(1/self.lambda_))  # wait a random entry time
            arrival_time = self.env.now
            
            print(f"Customer {id_num} arrived at {arrival_time:.2f}")

            # a customer arrives
            # simpy.Store automatically drops items when out of capacity
            # but in order to save data let's check it
            is_free_queue_space = (len(self.queue.items) < self.queue.capacity)
            if is_free_queue_space:
                # admit to queue and enter process for them
                
                # self.results.update({id_num: [self.env.now, None, None, True]})
                self.queue.put(id_num) # got into the queue 
                self.env.process(self.process_customer(id_num))
            else:
                # record their attempted entry but do not enter the queue
                # self.results.update({id_num: [self.env.now, None, None, False]})
                print(f"Customer {id_num} was rejected at {self.env.now:.2f}", "cur_cust", self.current_customer)

                #### data ####
        
            id_num += 1 
            self.current_customer = id_num

    
    def process_customer(self, id_num):
        # serve a customer at the front of the queue
        with self.server.request() as request:
            yield request # Вроде здесь как раз и ждем своей очереди на ресурс
            
            assert self.queue.items[0] == id_num, 'Wrong Customer ID got to the server'
            self.queue.get()
            print(f"Customer {id_num} went to barber at {self.env.now:.2f}", "cur_cust", self.current_customer)
            
            # self.results[id_num][1] = self.env.now  # time moved to the counter
            service_time = random.exponential(1/self.mu)
            yield self.env.timeout(service_time)
            print(f"Customer {id_num} got service and left at {self.env.now:.2f}","cur_cust", self.current_customer)
            
            # self.results[id_num][2] = self.env.now  # time left the system

    # def calculate_metrics(self, id_num):
    #     metrics[id_num] = {
            
    #     }

In [322]:
a = simulatiuon.server

In [384]:
d = pd.DataFrame(res)

In [397]:
env = simpy.Environment()
simulatiuon = MMcK_Queue(env, lambda_=lam, c=c,K=K, mu=mu)
res = simulatiuon.run_until_time(10)

Customer 0 arrived at 0.11
Customer 0 went to barber at 0.11
Customer 1 arrived at 0.13
Customer 1 went to barber at 0.13
Customer 2 arrived at 0.14
Customer 2 went to barber at 0.14
Customer 3 arrived at 0.21
Customer 1 got service and left at 0.22
Customer 3 went to barber at 0.22
Customer 4 arrived at 0.26
Customer 3 got service and left at 0.28
Customer 4 went to barber at 0.28
Customer 5 arrived at 0.28
Customer 0 got service and left at 0.30
Customer 5 went to barber at 0.30
Customer 6 arrived at 0.35
Customer 7 arrived at 0.38
Customer 2 got service and left at 0.44
Customer 6 went to barber at 0.44
Customer 5 got service and left at 0.48
Customer 7 went to barber at 0.48
Customer 7 got service and left at 0.51
Customer 8 arrived at 0.61
Customer 8 went to barber at 0.61
Customer 9 arrived at 0.69
Customer 10 arrived at 0.72
Customer 4 got service and left at 0.73
Customer 9 went to barber at 0.73
Customer 11 arrived at 0.81
Customer 6 got service and left at 0.88
Customer 10 we

In [262]:
simulatiuon.run_until_time(100)

Customer 0 arrived at 0.013744312309021805
Customer 0 got to barber at 0.013744312309021805
Customer 0 got service and left at 0.020233833377499538
Customer 1 arrived at 0.02123252264057144
Customer 1 got to barber at 0.02123252264057144
Customer 2 arrived at 0.058783679242699366
Customer 2 got to barber at 0.058783679242699366
Customer 3 arrived at 0.06608137668609632
Customer 3 got to barber at 0.06608137668609632
Customer 3 got service and left at 0.07116108409742852
Customer 4 arrived at 0.07462431172481339
Customer 4 got to barber at 0.07462431172481339
Customer 5 arrived at 0.1579641168372174
Customer 6 arrived at 0.21966539928941653
Customer 4 got service and left at 0.24560619427075303
Customer 5 got to barber at 0.24560619427075303
Customer 7 arrived at 0.2707765297893671
Customer 8 arrived at 0.2716565063811527
Customer 5 got service and left at 0.27209860050937584
Customer 6 got to barber at 0.27209860050937584
Customer 2 got service and left at 0.3951603444584437
Customer 7

In [117]:
class Car(object):
    def __init__(self, env):
        self.env = env
        self.charges_count = 0
        self.action = env.process(self.run())

    def run(self):
        while True:
            print('Start parking and charging at %d' % self.env.now)
            charge_duration = 5
            

            # We may get interrupted while charging the battery
            try:
                yield self.env.process(self.charge(charge_duration))
                self.charges_count += 1
            except simpy.Interrupt:
                # When we received an interrupt, we stop charging and
                # switch to the "driving" state
                print('Was interrupted. Hope, the battery is full enough ...')

            print('Start driving at %d' % self.env.now)
            trip_duration = 2
            yield self.env.timeout(trip_duration)

    def charge(self, duration):
        if self.env.now > 0 and self.charges_count % 4 == 0:
            yield self.env.timeout(2)
            # print("Driver's tired of waiting at {}".format(self.env.now) )
            # self.action.interrupt()
            self.action.interrupt()
        yield self.env.timeout(duration)

In [35]:
# def car(env):
#     while True:
#         print('Start parking at %d' % env.now)
#         parking_duration = 5
#         yield env.timeout(parking_duration)

#         print('Start driving at %d' % env.now)
#         trip_duration = 2
#         yield env.timeout(trip_duration)

In [242]:
l = [1, 2]
l.remove(1)

[2]

In [118]:
env = simpy.Environment()
car = Car(env)
env.run(until=40)

Start parking and charging at 0
Start driving at 5
Start parking and charging at 7
Start driving at 12
Start parking and charging at 14
Start driving at 19
Start parking and charging at 21
Start driving at 26
Start parking and charging at 28
Was interrupted. Hope, the battery is full enough ...
Start driving at 30
Start parking and charging at 32
Was interrupted. Hope, the battery is full enough ...
Start driving at 34
Start parking and charging at 36
Was interrupted. Hope, the battery is full enough ...
Start driving at 38


In [125]:
from numpy import random

In [248]:
# q.get

In [228]:
q.get_queue[0].

<StoreGet() object at 0x12f37ebf0>

In [155]:
# env2 = simpy.Environment()
# bcs = simpy.Resource(env2, capacity=2)

In [154]:
def car(env, name, bcs, avg_driving_time, charge_duration):
    # Simulate driving to the BCS
    yield env.timeout(random.exponential(scale=avg_driving_time))

    # Request one of its charging spots
    print('%s arriving at %d' % (name, env.now))
    with bcs.request() as req:
        yield req

        # Charge the battery
        print('%s starting to charge at %s' % (name, env.now))
        yield env.timeout(random.exponential(scale=charge_duration))
        print('%s leaving the bcs at %s' % (name, env.now))

In [156]:
# env2.process(car(env2, 1, bcs, 1/lam, 1/mu))
# env2.run(until=40)